In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy.integrate as integrate
import ipyvolume as ipv
from importlib import reload

import os
import sys
sys.path.insert(0, os.path.abspath('pycode'))

In [ ]:
importlib.reload(ipv)

In [ ]:
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111)

def noise(x):
    return np.sin(x)

def fbm(octaves, lacunarity, gain, x):
    # initial values
    amplitude = 0.5;
    frequency = 1.;

    #  loop of octaves
    y = np.zeros_like(x)
    for i in range(int(octaves)):
        y += amplitude * noise(frequency*x);
        frequency *= lacunarity;
        amplitude *= gain;
    return y

def plot_fbm(octaves, lacunarity, gain):
    x = np.arange(-10, 10, 0.1)
    y = fbm(octaves, lacunarity, gain, x)
    ax.clear()
    ax.plot(x, y)

widgets.interact(plot_fbm, octaves=(1, 10, 1.0), lacunarity=(0, 10.0, 0.1), gain=(0, 2, 0.1))

In [ ]:
import sampling as sampling
import spherical as sph
import BRDF as BRDF
import LTC as LTC

import mitsuba.python.chi2 as chi2

import importlib
importlib.reload(sampling)
importlib.reload(sph)
importlib.reload(BRDF)
importlib.reload(LTC)

# samples_ = np.random.uniform(size =[2,1024*32])
# ggx = sampling.NormalDistributionGGX()
# alpha = 0.5
# Wh = ggx.sample(alpha, samples_)
# pdf = ggx.pdf(alpha, Wh)
# D = ggx.eval(alpha, Wh)


ltc = LTC.LTC(0.2, 0.2, 1)

def my_sample(sample):
    return ltc.sample(sample)
    return ggx.sample(alpha, sample)
    return square_to_cosine_hemisphere(sample)

def my_pdf(p):
    return ltc.pdf(p)
    return ggx.pdf(alpha, p)
    return square_to_cosine_hemisphere_pdf(p)

chi2Test = chi2.ChiSquareTest(
    domain=chi2.SphericalDomain(),
    sample_func=my_sample,
    pdf_func=my_pdf,
    sample_dim=2,
    sample_count=1000000,
    ires=32
)

chi2Test.tabulate_histogram()
chi2Test.tabulate_pdf(simpson_rule=True)
chi2Test.run()

fig, ax = plt.subplots(2)
fig.set_figwidth(8)
fig.set_figheight(8)
vmax = np.maximum(np.amax(chi2Test.histogram), np.amax(chi2Test.pdf))
# vmax *= 0.96
ax[0].set_title('Observed frequencies')
ax[0].imshow(chi2Test.histogram, cmap='jet', vmax=vmax)
ax[1].set_title('Expected frequencies')
ax[1].imshow(chi2Test.pdf, cmap='jet', vmax=vmax)
# print(chi2Test.histogram.shape)

# vmax = np.amax(chi2Test.histogram)
# vmax = np.maximum(vmax, np.amax(chi2Test.pdf))
# plt.imshow(chi2Test.histogram, cmap='jet', vmax=vmax)
# print(chi2Test.histogram.shape)
# # print(chi2Test.histogram.shape)
# # chi2.run()
# # chi2._dump_tables()

In [ ]:
print(vmax)
print(np.amax(chi2Test.histogram))
print(np.amax(chi2Test.pdf))

In [ ]:

import LTC as LTC
import spherical as sph
importlib.reload(LTC)
importlib.reload(sph)

ltc = LTC.LTC()
theta, phi = sph.meshgrid_spherical_coord(128)
Wi = sph.spherical_dir(theta, phi)
D = ltc.eval(Wi) * np.pi

In [ ]:
import os
import numpy as np
import mitsuba

# Set the desired mitsuba variant
mitsuba.set_variant('scalar_rgb')

from mitsuba.core import Bitmap, Struct, Thread
from mitsuba.core.xml import load_file

# Absolute or relative path to the XML file
filename = 'W:/Mitsuba/mitsuba2/resources/data/scenes/cbox/cbox.xml'

# Add the scene directory to the FileResolver's search path
Thread.thread().file_resolver().append(os.path.dirname(filename))

# Load the actual scene
scene = load_file(filename)

# Call the scene's integrator to render the loaded scene
scene.integrator().render(scene, scene.sensors()[0])

# After rendering, the rendered data is stored in the film
film = scene.sensors()[0].film()

# Write out rendering as high dynamic range OpenEXR file
film.set_destination_file('W:/Mitsuba/mitsuba2/resources/data/scenes/cbox/output.exr')
film.develop()

# Write out a tonemapped JPG of the same rendering
bmp = film.bitmap(raw=True)
bmp.convert(Bitmap.PixelFormat.RGB, Struct.Type.UInt8, srgb_gamma=True).write('W:/Mitsuba/mitsuba2/resources/data/scenes/cbox/output.jpg')

# Get linear pixel values as a numpy array for further processing
bmp_linear_rgb = bmp.convert(Bitmap.PixelFormat.RGB, Struct.Type.Float32, srgb_gamma=False)
image_np = np.array(bmp_linear_rgb)
print(image_np.shape)


In [ ]:
import numpy as np
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_rgb')
from mitsuba.core import Float

import BRDF
import spherical as sph

reload(BRDF)
reload(sph)

import ipyvolume as ipv

alpha = ek.linspace(Float, BRDF.MIN_GGX_ALPHA, 1, 10)
theta = ek.linspace(Float, 0, ek.pi/2, 10)
err0 = np.zeros((10, 10))
err1 = np.zeros((10, 10))

for i in range(len(alpha)):
    for j in range(len(theta)):
        Wo = sph.spherical_dir(theta[j], 0)
#         err0[i][j] = BRDF.maskingFuncProjectedAreaTest(BRDF.NDF_GGX_, BRDF.smithMaskingFunction, alpha[i], Wo, num_samples=1024)[0]
        err1[i][j] = BRDF.whiteFurnaceTest(BRDF.NDF_GGX_, BRDF.smithMaskingFunction, alpha[i], Wo, num_samples=4096)[0]

alpha = alpha.numpy()
theta = theta.numpy()
alpha, theta = np.meshgrid(alpha, theta)
ipv.clear()
ipv.plot_mesh(alpha, theta, err1)
ipv.show()

